# 👨‍💼👨‍🚀👨‍💻 Agents with Plugins

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.0-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    //.AddOpenAIChatCompletion(
    //    modelId: Secrets.OpenAIModel,
    //    apiKey: Secrets.OpenAIApiKey)    
    .Build();

### Let's equip the kernel with the Alerts plugin

In [ ]:
#!import ../../Plugins/AlertsPlugin.cs

kernel.Plugins.AddFromType<AlertsPlugin>();

### 🧱 A persona is the behaviour we give to an agent. Our newly created agent gets the behavior of Jack Sparrow pirate.

In [ ]:
#pragma warning disable SKEXP0110

ChatCompletionAgent jackSparrowAgent = new() 
{ 
    Arguments = new KernelArguments (new OpenAIPromptExecutionSettings { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions }),
    Kernel = kernel,
    Instructions = """
        You are Jack Sparrow talking in Jack Sparrow style.
        Evaluate the context and reply to the last message by providing exactly one meaningful dialog line.
        The dialog line must be only one sentence of maximum 10 words.
        """,
    Description = "A chat bot that replies to the message in the voice of Jack Sparrow talking style.",
    Name = "JackSparrow",
    Id = "JackSparrow_01",
};

### 🧱 Let's make another agent. This one impersonates Yoda.

In [ ]:
#pragma warning disable SKEXP0110

ChatCompletionAgent yodaAgent = new()
{
    Arguments = new KernelArguments (new OpenAIPromptExecutionSettings { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions }),
    Kernel = kernel,
    Instructions = """
        You are Yoda talking in Yoda style.
        Evaluate the context and reply to the last message by providing exactly one meaningful dialog line.
        The dialog line must be only one sentence of maximum 10 words.
        """,
    Description = "A chat bot that replies to the message in the voice of Yoda talking style.",
    Name = "Yoda",
    Id = "Yoda_01",
};

### See that the execution settings tool behaviour is set o autoinvoke kernel plugins!
### Both agents are connected to the same kernel, therefore they share the same plugins.

### Let's create a chat group for agents and add a goal to the chat.

In [ ]:
#pragma warning disable SKEXP0110

AgentGroupChat chat = new(jackSparrowAgent, yodaAgent)
{
    ExecutionSettings = new()
    {
        TerminationStrategy = new AggregatorTerminationStrategy(
            new RegexTerminationStrategy(@"\b(rum)\b")
            {
                Agents = [jackSparrowAgent]
            },
            new RegexTerminationStrategy(@"\b(tea)\b")
            {
                Agents = [yodaAgent]
            }
        )
        { 
            Condition = AggregateTerminationCondition.Any,
            MaximumIterations = 10
        },
        SelectionStrategy = new SequentialSelectionStrategy
        {
        },
    }
};

var goal = """
    Jack Sparrow makes a bad joke about Yoda's taste in drinks.
    """;

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, goal));


### ⚡Let the story (chat) begin!

In [ ]:
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

Console.WriteLine($">>>>>>> {AuthorRole.User} > {goal}.");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"\n>>>>>>> {content.Role} [{content.AuthorName}] > {content.Content}");
}

Console.WriteLine($"IS COMPLETE: {chat.IsComplete}");

### Notice that the chat history is invoked for all attached agents. Even the plugins were triggered, as well. 